### Install Dash

In [13]:
!pip3 install --upgrade plotly
!pip3 install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached plotly-5.11.0-py2.py3-none-any.whl (15.3 MB)
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output 
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
from google.colab import files

## Generate datasets

#### 1 parameter

In [15]:
def generate_dataset_1(link, column1):
  df = pd.read_csv(link) 
  
  df['date'] = df['date'].astype('datetime64[ns]')
  df['week'] = df['date'].dt.isocalendar().week
  df = df.drop(['date'], axis=1)

  df = df.groupby(['week', column1])['week'].count().to_frame('count').reset_index()
  df = df.sort_values(['week', column1])

  columns = df[column1].drop_duplicates().to_numpy()
  for week in range(2, 42):
    for col1 in columns:
        if ( len(df.loc[(df['week'] == week) & (df[column1] == col1)]) == 0):
            row = {'week': week, column1: col1, 'count': 0}
            df = df.append(row, ignore_index = True)
  
  df = df.sort_values(by=['week'])
  df['week'] = df['week'].apply(lambda week: datetime.date(2022, 1, 1) + relativedelta(weeks=+week))
  return df

#### 2 parameters

In [16]:
def generate_dataset_2(link, column1, column2):
  df = pd.read_csv(link) 
  
  df['date'] = df['date'].astype('datetime64[ns]')
  df['week'] = df['date'].dt.isocalendar().week
  df = df.drop(['date'], axis=1)

  df = df.groupby(['week', column1, column2])['week'].count().to_frame('count').reset_index()
  df = df.sort_values(['week', column1, column2])

  columns = df[[column1, column2]].drop_duplicates().to_numpy()
  for week in range(2, 42):
    for col1, col2 in columns:
        if ( len(df.loc[(df['week'] == week) & (df[column1] == col1) & (df[column2] == col2)]) == 0):
            row = {'week': week, column1: col1, column2: col2, 'count': 0}
            df = df.append(row, ignore_index = True)
  
  df = df.sort_values(by=['week'])
  df['week'] = df['week'].apply(lambda week: datetime.date(2022, 1, 1) + relativedelta(weeks=+week))
  return df

## Plot figure

#### 1 parameter

In [17]:
def plot_figure_1(link, column1):
  df = generate_dataset_1(link, column1)
  
  fig = px.line(df, x="week", y="count", color=column1, title='')
  fig.show()

#### 2 parameters

In [18]:
def plot_figure_2(link, column1, column2):
  df = generate_dataset_2(link, column1, column2)
  
  app = JupyterDash(__name__)

  app.layout = html.Div([	
      # html.H4('Count by attacker (line) vs attacked (checkbox)'),	
      dcc.Graph(id="graph"),	
      dcc.Checklist(	
          id="checklist",	
          options=sorted(df[column1].unique()),	
          value=["Ukraine"],	
          inline=True	
      ),	
  ])	

  @app.callback(	
      Output("graph", "figure"), 	
      Input("checklist", "value"))
  
  def update_line_chart(labels):	
      mask = df[column1].isin(labels)	
      fig = px.line(df[mask], 	
          x="week", y="count", color=column2)	
      return fig


  app.run_server(mode='inline', debug=True)

## Read

In [19]:
link = 'https://drive.google.com/uc?export=download&id=1yHGTAm0IyyUUHUchv6aAdBqYOdAGWHPu'

## Plot

### Count by attacked (line)

In [8]:
plot_figure_1(link, 'attacked_country')

### Count by attacker (line)

In [9]:
plot_figure_1(link, 'attacking_country')

### Download: Count by attacked (checkbox) vs attacker (line) 

In [10]:
filename = 'attacker_attacked.csv'

attacker_attacked = generate_dataset_2(link, 'attacked_country', 'attacking_country')
attacker_attacked.to_csv(filename)

files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Count by attacker (checkbox) vs attacked (line) 

In [11]:
plot_figure_2(link, 'attacking_country', 'attacked_country')

<IPython.core.display.Javascript object>

### Count by attacking organisation (checkbox) vs attack type (line) 

In [12]:
filename = 'organisation_type.csv'

organisation_type = generate_dataset_2(link, 'attacking_organisation', 'attack_type')
organisation_type.to_csv(filename)

files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>